In [ ]:
import numpy as np
import math
import sys
import statistics
import tensorflow as tf
import string
import random
import matplotlib.pyplot as plt
import glob
import os
import cv2
from PIL import Image
from tensorflow import keras
from tensorflow.keras import layers
from google.colab import drive
from tensorflow.python.framework import ops
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.models import Model
from tensorflow.keras.layers.experimental import preprocessing

drive.mount('/content/gdrive')

# NN for data augmentation
data_augmentation = tf.keras.Sequential([
    layers.experimental.preprocessing.RandomWidth(0.1),
    layers.experimental.preprocessing.RandomHeight(0.1),
    layers.experimental.preprocessing.RandomZoom(0.2),
  ])

d = tf.keras.models.load_model('/content/gdrive/My Drive/NewUnsupFace')
c = tf.keras.models.load_model('/content/gdrive/My Drive/NewUnsupCNN')
modelDen = keras.models.Sequential()
modelCNN = keras.models.Sequential()

# all of this is hard coded. Sadness.
modelDen.add(d.layers[0])
modelDen.add(keras.layers.Dense(16834))
modelDen.add(keras.layers.Activation('relu'))
modelDen.add(keras.layers.Dropout(0.6))
modelDen.add(keras.layers.Dense(2))  # fake or real face
modelDen.add(keras.layers.Activation('sigmoid'))


# CNN
modelCNN.add(c.layers[0])
modelCNN.add(keras.layers.Flatten())
modelCNN.add(keras.layers.Dense(16834))
modelCNN.add(keras.layers.Activation('relu'))
modelCNN.add(keras.layers.Dropout(0.6))
modelCNN.add(keras.layers.Dense(2))  # fake or real face
modelCNN.add(keras.layers.Activation('sigmoid'))

x_train = []
x_test = []

y_train = []
y_test = []

x = []
y = []

realCount = 0
fakeCount = 0

print("Getting real faces")
for filename in glob.iglob(os.path.join('/content/gdrive/My Drive/realAndFake/real_and_fake_face/training_real/', "*.jpg")):
  img = cv2.imread(filename)
  img = cv2.resize(img, (128,128))
  img = img_to_array(img) 
  img = tf.image.rgb_to_grayscale(img)
  x.append(img)
  y.append([1])
  realCount += 1



print("Getting fake faces")
for filename in glob.iglob(os.path.join('/content/gdrive/My Drive/realAndFake/real_and_fake_face/training_fake/', "*.jpg")):
  img = cv2.imread(filename)
  img = cv2.resize(img, (128,128))
  img = img_to_array(img) 
  img = tf.image.rgb_to_grayscale(img)
  x.append(img)
  y.append([0])
  fakeCount += 1


x_train = x[:1428]
x_test = x[1428:]

y_train = y[:1428]
y_test = y[1428:]

y_test = ops.convert_to_tensor(y_test, dtype=tf.float32) / 255.00
y_train = ops.convert_to_tensor(y_train, dtype=tf.float32) / 255.00
x_train = tf.convert_to_tensor(x_train, dtype=tf.float32) / 255.00
x_test = tf.convert_to_tensor(x_test, dtype=tf.float32) / 255.00


modelDen.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss=keras.losses.MeanSquaredError(), metrics='accuracy')
modelDen.fit(x=x_train, y=y_train, shuffle=True, batch_size=128, epochs=25, validation_data=(x_test, y_test))
modelDen.save('/content/gdrive/My Drive/SavedUnSupModel')

print("================================================")
print("================================================")
print("================================================")
print("Training CNN model now.")

modelCNN.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss=keras.losses.SparseCategoricalCrossentropy(), metrics='accuracy')
modelCNN.fit(x=x_train, y=y_train, shuffle=True, batch_size=128, epochs=25, validation_data=(x_test, y_test))
modelCNN.save('/content/gdrive/My Drive/SavedUnSupCNN')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Getting real faces
Getting fake faces
Epoch 1/25
12/12 [==============================] - 3s 183ms/step - loss: 0.2458 - accuracy: 0.2430 - val_loss: 0.2470 - val_accuracy: 1.0000
Epoch 2/25
12/12 [==============================] - 2s 172ms/step - loss: 0.2428 - accuracy: 0.2430 - val_loss: 0.2440 - val_accuracy: 1.0000
Epoch 3/25
12/12 [==============================] - 2s 168ms/step - loss: 0.2399 - accuracy: 0.2430 - val_loss: 0.2411 - val_accuracy: 1.0000
Epoch 4/25
12/12 [==============================] - 2s 165ms/step - loss: 0.2370 - accuracy: 0.2430 - val_loss: 0.2382 - val_accuracy: 1.0000
Epoch 5/25
12/12 [==============================] - 2s 168ms/step - loss: 0.2341 - accuracy: 0.2430 - val_loss: 0.2353 - val_accuracy: 1.0000
Epoch 6/25
12/12 [==============================] - 2s 167ms/step - loss: 0.2312 - accuracy: 0.2430 - val_loss: 0.2324 - 